# Zoom 기능과 출석체크 기능 추가

- a 입력 : 출석한 인원 확인후 출력
- 1, 2, 3, 4, \` 입력 : n번 스크린으로 화면 전환(\`은 기본화면)
- q 입력 : 종료

In [3]:
import face_recognition
import cv2
import numpy as np
import os, glob
import urllib.request

# This is a demo of running face recognition on live video from your webcam. It's a little more complicated than the
# other example, but it includes some basic performance tweaks to make things run a lot faster:
#   1. Process each video frame at 1/4 resolution (though still display it at full resolution)
#   2. Only detect faces in every other frame of video.

# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

# Get a reference to webcam #0 (the default one)

def load_img(file_path):
    img_path = []
    dir_path = os.path.join(os.getcwd(), file_path)
    folder_path = glob.glob(os.path.join(dir_path, '*'))
    for _ in folder_path:
        img_path.append(glob.glob(os.path.join(_, '*')))
    return img_path

def size_up(img, size):
    dst = cv2.resize(img, dsize=(0, 0), fx=size, fy=size, interpolation=cv2.INTER_CUBIC)
    return dst

def mod_crop(image, scale = 2):
    if len(image.shape) ==3:
        h = image.shape[0]
        w = image.shape[1]
        h = h - np.mod(h,scale)
        w = w - np.mod(w,scale)
        return image[0:h,0:w,:]

def face_names(file_path):
    face_names = []
    dir_path = os.path.join(os.getcwd(), file_path)
    img_path = glob.glob(os.path.join(dir_path, '*'))
    for _ in img_path:
        face_names.append(_.split('\\')[-1])
    return face_names

def face_encoding(img_path):
    # Load a sample picture and learn how to recognize it.
    temp = []
    for folder in img_path:
        for _ in folder:    
            face_image = face_recognition.load_image_file(_)
            result = face_recognition.face_encodings(face_image)[0]
        temp.append(result)
    return temp

known_face_encodings = face_encoding(load_img('image/face_recognition/train_data'))
known_face_names = face_names('image/face_recognition/train_data')

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
attendance_list = []
screen = 0

URL = "http://192.168.43.1:8080/shot.jpg"
while True:
    try:
        img_arr = np.array(bytearray(urllib.request.urlopen(URL).read()),dtype=np.uint8)
        frame = cv2.imdecode(img_arr,-1)
        if cv2.waitKey(1) & 0xFF == ord('a'):            
            cam = [frame]
            # Display the resulting image
            cv2.putText(frame,"WAIT...", (0, 100), cv2.FONT_ITALIC, 3, (255, 0, 255), 3)        
            cv2.imshow('Video', frame)
            img = mod_crop(frame, 10)
            height = img.shape[0]
            width = img.shape[1]
            channel = img.shape[2]
            # 12개의 화면으로 분할하고 각각의 화면을 3배확대하여 저장함
            for H_1, H_2 in [(0,int(height*0.4)), (int(height*0.3),int(height*0.7)), (int(height*0.6),height)]:
                for W_1, W_2 in [(0,int(width*0.3)), (int(width*0.2),int(width*0.5)), (int(width*0.4),int(width*0.7)), (int(width*0.6),width)]:
                    cam.append(size_up(img[H_1:H_2,W_1:W_2],3))
            
            face_names = []    
            for frame in cam:
                # Resize frame of video to 1/4 size for faster face recognition processing
                small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
                
                # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
                rgb_small_frame = small_frame[:, :, ::-1]

                # Find all the faces and face encodings in the current frame of video
                face_locations = face_recognition.face_locations(rgb_small_frame)
                face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)               
                for face_encoding in face_encodings:
                    # See if the face is a match for the known face(s)
                    matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                    name = "Unknown"
                    # Or instead, use the known face with the smallest distance to the new face
                    face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                    best_match_index = np.argmin(face_distances)
                    min_val = min(face_distances)
                    if matches[best_match_index]:
                         if min_val < 0.5:
                                name = known_face_names[best_match_index]
                    face_names.append(name)
            attendance_list = list(set(face_names))
            print(attendance_list)
            continue
        
        # 사용할 스크린의 번호를 입력
        if cv2.waitKey(1) & 0xFF == ord('1'):
            height = frame.shape[0]
            width = frame.shape[1]
            channel = frame.shape[2]
            screen = 1
        elif cv2.waitKey(1) & 0xFF == ord('2'):  
            height = frame.shape[0]
            width = frame.shape[1]
            channel = frame.shape[2]
            screen = 2
        elif cv2.waitKey(1) & 0xFF == ord('3'):  
            height = frame.shape[0]
            width = frame.shape[1]
            channel = frame.shape[2]
            screen = 3           
        elif cv2.waitKey(1) & 0xFF == ord('4'):  
            height = frame.shape[0]
            width = frame.shape[1]
            channel = frame.shape[2]
            screen = 4
        elif cv2.waitKey(1) & 0xFF == ord('`'):  
            height = frame.shape[0]
            width = frame.shape[1]
            channel = frame.shape[2]
            screen = 0
        
        if screen == 1:
            frame = size_up(frame[0:int(height*0.5),0:int(width*0.5)],2)
        elif screen == 2:
            frame = size_up(frame[0:int(height*0.5),int(width*0.5):width],2)
        elif screen == 3:
            frame = size_up(frame[int(height*0.5):height,0:int(width*0.5)],2)
        elif screen == 4:
            frame = size_up(frame[int(height*0.5):height,int(width*0.5):width],2)      
            
            
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]

        # Only process every other frame of video to save time
        if process_this_frame:
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                # See if the face is a match for the known face(s)
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"
                # Or instead, use the known face with the smallest distance to the new face
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                min_val = min(face_distances)
                if matches[best_match_index]:
                     if min_val < 0.5:
                            name = known_face_names[best_match_index]

                face_names.append(name)

        process_this_frame = not process_this_frame


        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        # Display the resulting image
        if screen == 1:
            cv2.putText(frame,"Screen 1", (0, 100), cv2.FONT_ITALIC, 2, (255, 0, 255), 3)    
        elif screen == 2:
            cv2.putText(frame,"Screen 2", (0, 100), cv2.FONT_ITALIC, 2, (255, 0, 255), 3)    
        elif screen == 3:
            cv2.putText(frame,"Screen 3", (0, 100), cv2.FONT_ITALIC, 2, (255, 0, 255), 3)    
        elif screen == 4:
            cv2.putText(frame,"Screen 4", (0, 100), cv2.FONT_ITALIC, 2, (255, 0, 255), 3)    
        cv2.imshow('Video', frame)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    except Exception as e:
        print("Error : ", e)
        break

# Release handle to the webcam
cv2.destroyAllWindows()

[]
[]
['JS']
[]
[]
['Unknown']
[]
['Unknown', 'JS']
[]
[]
[]
['JS']
['JS']
['Unknown', 'JS']
['Unknown']
['JS']
['JS']
Error :  OpenCV(4.1.0) C:\projects\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:726: error: (-215:Assertion failed) !buf.empty() && buf.isContinuous() in function 'cv::imdecode_'

